# Röntgenspektrum


## Einleitung

In diesem Dokument sind die Schritte für die Steuerung des Röntgenspektrometers aufgeführt.

## Hinweise zum Umgang mit Jupyter-Notebook

Das Experiment wird mittels eines Jupyter-Notebooks gesteuert. Für die Nutzung werden in diesem Abschnitt einige Hinweise gegeben. 

### Allgemein
Bei Jupyter-Notebook handelt es sich um eine Schnittstelle zu einem Python-Prozess.

Der Code wird dabei nacheinander in Zellen ausgeführt. Alternativ lassen sich in den Zellen Schritte kommentieren und Gleichungen aufschreiben. Dafür wird Markdown (https://markdown.de/), LaTeX-Syntax (für Gleichungen bspw. $E=mc^2$) oder HTML verwendet.

Eine neue Zelle kann über den Button <img src="QA_neue_Zelle.png"> erstellt werden.

Der in einer Zelle aufgeführte Code wird mit ``SHIFT`` + ``ENTER`` ausgeführt. Die dabei erstellten Variablen bleiben für weitere Rechenschritte erhalten.

Einige weitere nützliche Tastenkombinationen sind in der folgenden Tabelle aufgeführt.


Funktion | Tastenkombination
      :--|:---
neue Zelle oberhalb einfügen | ``a`` (im Navigationsmodus)
neue Zelle unterhalb einfügen | ``b`` (im Navigationsmodus)
Löschen einer Zelle | doppelt ``d`` (im Navigationsmodus)
Ausführen einer Zelle | ``SHIFT`` + ``ENTER``

Für weitere Informationen zu Jupyter-Notebook sei beispielsweise auf die folgenden Internetseiten verwiesen:
* https://realpython.com/jupyter-notebook-introduction/
* https://jupyter-notebook.readthedocs.io/en/stable/index.html

### Herunterladen von Messdaten
Das Kopieren der Daten vom Laborcomputer auf den eigenen Computer kann direkt über den Browser geschehen, falls der Zugriff auf den Laborcomputer über das Netzwerk geschieht:
* ``File`` $\rightarrow$ ``Open...``
* zur Datei navigieren
* Datei mit Kästchen auswählen
* Download

## Vorbereitung

### Installationen auf dem Laborcomputer

Die folgenden Schritte müssen bei der Neueinrichtung des Labrcomputers für das Experiment durchgeführt werden. Bei Verwendung der Anaconda Distribution können die Pakete entsprechend über conda installiert werden.

* Jupyter-Notebook installieren
    * ``sudo pip3 install --upgrade pip``
    * ``sudo pip3 install jupyter``

* Sounddevice installieren
    * ``pip3 install sounddevice``
    * Download PortAudio http://files.portaudio.com/download.html
    * ``cd portaudio``
    * ``./configure``
    * ``sudo make install``
    * ``sudo apt-get install libportaudio2``

* PySerial installieren
    * ``pip3 install pyserialv

* Weitere Bibliotheken installieren
    * ``pip3 install matplotlib``
    * ``pip3 install numpy``
    * ``pip3 install scipy``

### Schrittmotorsteuerung

In [ ]:
# Kommunikation mit serieller Schnittstelle
import time
import serial
from serial.tools import list_ports

# Auslesen und Freimachen des Buffers
# Optional kann der Inhalt ausgegeben werden
def read_complete_buffer(ser,printing=False):
    while(True):
        result = ser.readline()
        if result == b'':
            break
        if printing:
            print(result)

# Schritte mit dem Schrittmotor gehen
def go(ser,steps):
    ser.write(bytes(str(steps),'utf-8'))
    time.sleep(1)
    read_complete_buffer(ser,True)
    
# Position abfragen
def position(ser):
    read_complete_buffer(ser,False)
    ser.write(bytes('pos','utf-8'))
    time.sleep(1)
    ser.readline()
    result = ser.readline()
    result = result[9:-2]
    print(result)
    return float(result)
    
# Microstepping einstellen
#  m1 - Setzt den Motor auf ganze Schritte
#  m2 - Halbschritte
#  m4 - Viertelschritte
#  m8 - Achtelschritte
# m16 - Sechzehntelschritte
def microstep(ser,string):
    ser.write(bytes(string,'utf-8'))
    time.sleep(1)
    read_complete_buffer(ser,True)

In [ ]:
# Serielle Schnittstellen anzeigen
schnittstellen = list_ports.comports() 
print('Index \tName')
for i in range(len(schnittstellen)):
    print(i,'\t',schnittstellen[i].device)

In [ ]:
# Auswählen der Schnittstelle über den Index
index = 0
# Herstellen der Verbindung
ser = serial.Serial(schnittstellen[index].device, 9600,timeout=1)

In [ ]:
# Größe der Mikroschritte wählen, hier muss ggf. probiert werden, bei welchen Mikroschritten der Motor
# im entsprechenden Setup am flüssigsten läuft.
microstep(ser,'m1')

In [ ]:
# Motor testweise laufen lassen
go(ser,5)

### Zähler
Das Zähler der Counts des Zählrohrs erfolgt über eine USB-Soundkarte und die Bibliothek Sounddevice (https://python-sounddevice.readthedocs.io/). Der Zähler wird dafür mit dem Mikrophoneingang der USB-Soundkarte verbunden.

<img src="Aufbau_Zaehler.png">

In [ ]:
import sounddevice as sd

Zu Beginn können die vorhandenen Audio-Schnittstellen angezeigt werden:

In [ ]:
sd.query_devices()

Die USB Sounkarte ``USB Audio Device`` oder ähnlich wird nun entsprechend ihrer Nummer ausgewählt. Die Nummer muss in folgender Zelle ggf. angepasst werden.

In [ ]:
# Hier wird die Soundkarte ausgewählt. Bitte den Index aus der vorher ausgegebenen Liste wählen.
sd.default.device = 2

In [ ]:
# Hier werden Samplerate und Anzahl der Kanäle festgelegt. (hier nichts ändern)
sd.default.samplerate = 44100
sd.default.channels = 1

In [ ]:
# Funktion zum Zählen von Impulsen in einem Array 
def count_array(array,threshold, deadSamples):
    counter = 0
    positions = []
    index = 0
    while index < len(array):
        if array[index]>=threshold:
            counter += 1
            positions.append(index)
            index += deadSamples
        else:
            index += 1
    return counter, positions

# kompakte Funktion zum Zählen
def count(duration):
    fs = 44100
    myrecording = sd.rec(int(duration * fs))
    sd.wait()
    
    # die Parameter threshold und deadSamples müssen angepasst werden (count_array(array,threshold, deadSamples))
    # dazu eine Aufnahme im Vorfeld machen und plotten: plt.plot(sd.rec(int(duration * fs)))
    counter, _ = count_array(myrecording,0.05,10)
    return counter

## Umsetzung der Messungen durch die Gruppen
Für die Speicherung der Daten einen Unterordner mit der Gruppennummer anlegen und dort abspeichern.
Bitte in einer Kopie dieser Datei arbeiten und die Kopie ebenfalls im Gruppenordner abspeichern.

In [ ]:
# Bespielhafter Scan Ablauf
duration = 10 # in s
stepsize = 1
number_of_steps = 100

positions = []
counts = []

for i in range(number_of_steps):
    print(i)
    counts.append(count(duration))
    go(ser,stepsize)
    positions.append(position(ser))
    print(i,counts[i],positions[i])

In [ ]:
plt.plot(positions,counts)